In [1]:
import matplotlib.pyplot as plt
import numpy as np
from astropy.visualization import astropy_mpl_style, quantity_support
import astropy.units as u
from astropy.coordinates import AltAz, EarthLocation, SkyCoord, ICRS
from astropy.time import Time

## Load array positions

In [2]:
desired_array = 'Mid' #(or 'Low')
if desired_array == 'Mid':
    array_x = [1.0]
    array_y = [2.0]
    array_z = [3.0]
    
    geographic_location = ''
    
elif desired_array == 'Low':
    print('FILL THIS IN ADAM')
else:
    print("That is not an SKA Telescope ")

## Set Target Coordinate(s) 
Give target coordinate(s) in ICRS

_single source only supported so far_

In [5]:
target = SkyCoord(frame=ICRS, ra = '19:34:23.5000', dec='-60:00:00.000', unit=(u.hourangle, u.deg))